# Market Sentiment with Alpha Vantage

### Setup

In [8]:
import os, sys, requests, datetime as dt
from dotenv import load_dotenv

load_dotenv()

BASE = "https://www.alphavantage.co/query"

def get_news():
    api_key = os.getenv("ALPHAVANTAGE_API_KEY")
    if not api_key:
        print("Missing alphavantage api key.")
        sys.exit(1)

    tickers = ["AAPL"]
    tickers_str = ",".join(tickers[:20])  # keep under 25 or so while on free tier
    # last 7 days, latest first, cap at 25 while on free tier
    time_from = (dt.datetime.utcnow() - dt.timedelta(days=7)).strftime("%Y%m%dT%H%M")

    params = {
        "function": "NEWS_SENTIMENT",
        "tickers": tickers_str,
        "time_from": time_from,
        "sort": "LATEST",
        "limit": 25,
        "apikey": api_key,
    }

    print(f"→ Hitting NEWS_SENTIMENT for: {tickers_str}")
    r = requests.get(BASE, params=params, timeout=30)
    print(f"HTTP {r.status_code}")
    try:
        data = r.json()
    except Exception:
        print("Non-JSON response:", r.text[:500])
        sys.exit(2)

    # Alpha Vantage returns 'Note' on throttling; 'Information' or 'Error Message' on issues
    for key in ("Note", "Information", "Error Message"):
        if key in data:
            print(f"{key}: {data[key]}")
            print("Rate-limit hit or endpoint not available")
            sys.exit(3)

    feed = data.get("feed", [])
    print(f"Articles returned: {len(feed)}")
    if not feed:
        print("No articles found")
        sys.exit(0)

    # Compact sample of the first few items
    for i, item in enumerate(feed[:5], start=1):
        ts_list = item.get("ticker_sentiment", [])
        tickers_in_item = [ts.get("ticker") for ts in ts_list]
        print(f"\n#{i}")
        print("published :", item.get("time_published"))
        print("source    :", item.get("source"))
        print("title     :", (item.get("title") or "")[:140])
        print("url       :", item.get("url"))
        print("mentions  :", tickers_in_item)
        if ts_list:
            ts = ts_list[0]
            print("av_sent   :", ts.get("ticker_sentiment_score"), "relevance:", ts.get("relevance_score"))

    print("\nSUCCESS: NEWS_SENTIMENT returned data with your key.")


### Grab news from AlphaVantage

In [9]:
get_news()


/var/folders/8r/txvrry2d4g18dhf7jkg1hhzm0000gn/T/ipykernel_67103/676731858.py:17: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  time_from = (dt.datetime.utcnow() - dt.timedelta(days=7)).strftime("%Y%m%dT%H%M")


→ Hitting NEWS_SENTIMENT for: AAPL
HTTP 200
Articles returned: 50

#1
published : 20251022T173926
source    : Benzinga
title     : What's Going On With The Drop In Apple Stock Today? - Apple  ( NASDAQ:AAPL ) 
url       : https://www.benzinga.com/trading-ideas/movers/25/10/48361867/whats-going-on-with-the-drop-in-apple-stock-today
mentions  : ['AAPL', 'FOREX:EUR']
av_sent   : -0.069515 relevance: 0.702807

#2
published : 20251022T172400
source    : Zacks Commentary
title     : The Mag 7 Stock Charts: Which are Hot?
url       : https://www.zacks.com/commentary/2774466/the-mag-7-stock-charts-which-are-hot
mentions  : ['MSFT', 'GOOG', 'META', 'NVDA', 'AAPL', 'TSLA', 'AMZN']
av_sent   : 0.247456 relevance: 0.511256

#3
published : 20251022T172100
source    : Zacks Commentary
title     : MSFT vs. AAPL: Which Mega-Cap Tech Stock is the Better Buy Now?
url       : https://www.zacks.com/stock/news/2774470/msft-vs-aapl-which-mega-cap-tech-stock-is-the-better-buy-now
mentions  : ['MSFT', 'GOOG', 